In [13]:
%run ./dataset.ipynb # runs file in context of current kernel, workaround for local import

In [20]:
import numpy as np
from torch import optim, nn, cuda
import tqdm

ImportError: cannot import name 'Variable' from 'torch' (/home/rock/.local/lib/python3.7/site-packages/torch/__init__.py)

In [15]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.init_size = 64 // 4
#       self.init_size = opt.img_size // 4
        self.l1 = nn.Sequential(nn.Linear(100, 128 * self.init_size ** 2))

        self.conv_blocks = nn.Sequential(
            nn.BatchNorm2d(128),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 128, 3, stride=1, padding=1),
            nn.BatchNorm2d(128, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 64, 3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 3, 3, stride=1, padding=1),
            nn.Tanh(),
        )

    def forward(self, z):
        out = self.l1(z)
        out = out.view(out.shape[0], 128, self.init_size, self.init_size)
        img = self.conv_blocks(out)
        return img

In [29]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        def discriminator_block(in_filters, out_filters, bn=True):
            block = [nn.Conv2d(in_filters, out_filters, 3, 2, 1), nn.LeakyReLU(0.2, inplace=True), nn.Dropout2d(0.25)]
            if bn:
                block.append(nn.BatchNorm2d(out_filters, 0.8))
            return block

        self.model = nn.Sequential(
            *discriminator_block(3, 16, bn=False),
            *discriminator_block(16, 32),
            *discriminator_block(32, 64),
            *discriminator_block(64, 128),
        )

        # The height and width of downsampled image
        ds_size = 64 // 2 ** 4
        self.adv_layer = nn.Sequential(nn.Linear(128 * ds_size ** 2, 2), nn.Sigmoid())

    def forward(self, img):
        out = self.model(img)
        out = out.view(out.shape[0], -1)
        validity = self.adv_layer(out)

        return validity

In [30]:
# Loss function
adversarial_loss = nn.CrossEntropyLoss()

gen = Generator()
disc = Discriminator()

optimizer_g = optim.Adam(gen.parameters(), lr=0.01, betas=(0.9,0.999))
optimizer_d = optim.Adam(disc.parameters(), lr=0.01, betas=(0.9,0.999))

image_iterator = GenerateIterator('./images_cropped')

In [32]:
start_epoch = 1

for epoch in range(start_epoch,101):
    progress_bar = tqdm.tqdm(image_iterator)
    for images in progress_bar:
        valid_image_labels = torch.ones(images.shape[0]).type(torch.LongTensor)
        fake_image_labels = torch.zeros(images.shape[0]).type(torch.LongTensor)
        seed = torch.from_numpy(np.random.normal(0,1, size=(images.shape[0], 100))).type(torch.FloatTensor)
        if cuda.is_available():
            valid_image_labels, fake_image_labels, images, seed = valid_image_labels.cuda(), fake_image_labels.cuda(), images.cuda(), seed.cuda()
            
        # --- train generator ---
        fake_images = gen(seed)
        gen_loss = adversarial_loss(disc(fake_images), valid_image_labels)
        
        optimizer_g.zero_grad()
        gen_loss.backward(retain_graph=True)
        optimizer_g.step()
        
        # --- train discriminator ---
        real_loss = adversarial_loss(disc(images), valid_image_labels)
        fake_loss = adversarial_loss(disc(fake_images), fake_image_labels)
        # divide by 2 or else loss is weighted 2x towards disc vs gen
        disc_loss = (real_loss + fake_loss) / 2
        
        optimizer_d.zero_grad()
        disc_loss.backward(retain_graph=True)
        optimizer_d.step()
        
        progress_bar.set_description(f"disc loss: {disc_loss} || gen loss: {gen_loss}")
        












  0%|          | 0/1287 [00:00<?, ?it/s]










disc loss: 0.693156361579895 || gen loss: 0.6931465864181519:   0%|          | 0/1287 [00:01<?, ?it/s]










disc loss: 0.693156361579895 || gen loss: 0.6931465864181519:   0%|          | 1/1287 [00:01<35:23,  1.65s/it]










disc loss: 0.6921187043190002 || gen loss: 0.6937211751937866:   0%|          | 1/1287 [00:03<35:23,  1.65s/it]










disc loss: 0.6921187043190002 || gen loss: 0.6937211751937866:   0%|          | 2/1287 [00:03<34:31,  1.61s/it]










disc loss: 0.692915141582489 || gen loss: 0.6974927186965942:   0%|          | 2/1287 [00:04<34:31,  1.61s/it] 










disc loss: 0.692915141582489 || gen loss: 0.6974927186965942:   0%|          | 3/1287 [00:04<33:31,  1.57s/it]










disc loss: 0.6904830932617188 || gen loss: 0.6465717554092407:   0%|          | 3/1287 [00:05<33:31,  1.57s/it]










disc loss: 0.6904830932617188 || gen loss: 0.6465717554092407:   0%|          | 4/1287 [00:05<3

disc loss: 0.8079582452774048 || gen loss: 0.665014922618866:   2%|▏         | 26/1287 [00:36<27:26,  1.31s/it] 










disc loss: 0.8079582452774048 || gen loss: 0.665014922618866:   2%|▏         | 27/1287 [00:36<27:14,  1.30s/it]










disc loss: 0.8466324806213379 || gen loss: 0.6592324376106262:   2%|▏         | 27/1287 [00:37<27:14,  1.30s/it]










disc loss: 0.8466324806213379 || gen loss: 0.6592324376106262:   2%|▏         | 28/1287 [00:37<26:44,  1.27s/it]










disc loss: 0.7883586883544922 || gen loss: 0.5969741940498352:   2%|▏         | 28/1287 [00:38<26:44,  1.27s/it]










disc loss: 0.7883586883544922 || gen loss: 0.5969741940498352:   2%|▏         | 29/1287 [00:38<26:02,  1.24s/it]










disc loss: 0.6671769618988037 || gen loss: 0.7408713102340698:   2%|▏         | 29/1287 [00:40<26:02,  1.24s/it]










disc loss: 0.6671769618988037 || gen loss: 0.7408713102340698:   2%|▏         | 30/1287 [00:40<25:47,  1.23s/it]










disc loss: 0.6522

disc loss: 0.778028130531311 || gen loss: 0.7103487253189087:   4%|▍         | 52/1287 [01:11<26:42,  1.30s/it] 










disc loss: 0.778028130531311 || gen loss: 0.7103487253189087:   4%|▍         | 53/1287 [01:11<26:56,  1.31s/it]










disc loss: 0.783348560333252 || gen loss: 0.6773485541343689:   4%|▍         | 53/1287 [01:12<26:56,  1.31s/it]










disc loss: 0.783348560333252 || gen loss: 0.6773485541343689:   4%|▍         | 54/1287 [01:12<26:40,  1.30s/it]










disc loss: 0.8826966285705566 || gen loss: 0.5665289163589478:   4%|▍         | 54/1287 [01:13<26:40,  1.30s/it]










disc loss: 0.8826966285705566 || gen loss: 0.5665289163589478:   4%|▍         | 55/1287 [01:13<25:59,  1.27s/it]










disc loss: 0.7829059362411499 || gen loss: 0.6891358494758606:   4%|▍         | 55/1287 [01:14<25:59,  1.27s/it]










disc loss: 0.7829059362411499 || gen loss: 0.6891358494758606:   4%|▍         | 56/1287 [01:14<26:20,  1.28s/it]










disc loss: 0.746156

KeyboardInterrupt: 